In [ ]:
import pandas as pd
import tweepy
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm.auto import tqdm


class twitter:
    api = None
    result_file = []
    trends_count = 0

    def __init__(self):
        # 키 수정
        consumer_key = "Your key"
        consumer_secret = "Your key"
        access_token = "Your key"
        access_secret = "Your key"

        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_secret)

        self.api = tweepy.API(auth, wait_on_rate_limit=True)
        print("twitter API connected!")

    def trends(self, url):
        # 여기 경로는 각자 selenium chrome path로 설정하면 됨. 동봉한 driver 참고
        driver = webdriver.Chrome(executable_path="./chromedriver.exe")
        # selenium이 웹 자원이 로드될 때 (3초)까지 기다려줌.
        driver.implicitly_wait(3)
        # chrome 페이지 open
        driver.get(url=url)
        # open한 페이지를 html화 -> bs로 긁어오기 위해
        html = driver.page_source
        # 위 코드도 가능하지만, lxml이 최근 버전에 호환이 더 잘 된다고 하고, 속도도 빠르다고 해서 lxml사용. 쓰려면 pip로 받아야 함.
        # soup = BeautifulSoup(html, 'html.parser')
        soup = BeautifulSoup(html, "lxml")
        trends = soup.select("td.main > a")
        # 따온 데이터를 text화 해서 각각 나타내기
        trends_arr = []
        for i in trends:
            tmp = i.get_text()
            trends_arr.append(tmp)
        # 창 닫기
        driver.close()
        return trends_arr
        print("twitter trends crawl done!")

    def init_work(self, trends: list):
        print("Start working...")

        for num, trend in enumerate(trends):
            self.search_keyword(trend)
            result = pd.DataFrame(
                self.result_file,
                columns=[
                    "keyword",
                    "posted_time",
                    "retweet count",
                    "text",
                    "primary words",
                ],
            )
            result.to_csv(str(num) + ".tsv", index=False, sep="\t")
            self.result_file = []

        print("Searching job done!!!")

    def search_keyword(self, keyword: str):
        print(f"keyword : {keyword} search start...")

        for tweet in tqdm(
            tweepy.Cursor(
                self.api.search, q=keyword, lang="ko", include_entities=False
            ).items(1000)
        ):
            if "RT" not in tweet.text:
                line = [keyword, str(tweet.created_at), tweet.text]
                self.result_file.append(line)

        print(f"keyword : {keyword} search done!\n")


In [ ]:
import pandas as pd

import trends_crawler

if __name__ == "__main__":
    url = "https://getdaytrends.com/ko/korea/top/tweeted/day/"
    total_trend = 0
    while True:
        print("What shall we do?\n1: crawl trends\n2: do colab\n3: merge all results")
        what_to_do = input()
        if what_to_do == "1":
            tweet = trends_crawler.twitter()
            trends_arr = tweet.trends(url)
            trends_arr = pd.DataFrame(trends_arr, columns=["hashtag"])
            trends_arr.to_csv("trends.tsv", index=False, sep="\t")
            print("csv file created!\ndone.\n")
        elif what_to_do == "2":
            tweet = trends_crawler.twitter()
            dataframe = pd.read_csv("trends.tsv")
            trends = []
            total_trend = len(dataframe) - 10
            for i in range(total_trend):
                trends.append(dataframe.iloc[i, 0])
            tweet.init_work(trends)
        elif what_to_do == "3":
            ds = pd.read_csv("0.tsv", delimiter="\t")
            for i in range(total_trend):
                ds1 = pd.read_csv(str(i) + ".tsv", delimiter="\t")
                ds = pd.concat([ds, ds1], ignore_index=True)
            ds.to_csv("result.tsv", index=False, sep="\t")
        else:
            break


What shall we do?
1: crawl trends
2: do colab
2
twitter API connected!
Start working...
keyword : #100MostHandsome2020 search start...



keyword : #100MostHandsome2020 search done!
keyword : #KimTaehyung search start...



keyword : #KimTaehyung search done!
keyword : #thetop_100 search start...



keyword : #thetop_100 search done!
keyword : #FandomoftheYear search start...



keyword : #FandomoftheYear search done!
keyword : #BTSARMY search start...


...


keyword : #Hopin_JunghwaxHwasa search done!
keyword : #Moonbyul_MiracleMV search start...



keyword : #Moonbyul_MiracleMV search done!
keyword : #Panorama4thWin search start...



keyword : #Panorama4thWin search done!
keyword : #AMiracleIsHere search start...



keyword : #AMiracleIsHere search done!
keyword : #우리는_끝까지_스토킹처벌법을_외친다 search start...



keyword : #우리는_끝까지_스토킹처벌법을_외친다 search done!
Searching job done!!!
What shall we do?
1: crawl trends
2: do colab
.
